# IMPORTS

## Libraries

In [1]:
import warnings

import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

from skopt                           import forest_minimize
from lightgbm                        import LGBMClassifier
from scipy.sparse                    import hstack, csr_matrix
from sklearn.metrics                 import roc_auc_score, average_precision_score
from sklearn.ensemble                import RandomForestClassifier
from sklearn.linear_model            import LogisticRegression
from sklearn.preprocessing           import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer


np.random.seed(0)
%matplotlib inline
pd.set_option('display.max_columns', 200)
warnings.filterwarnings("ignore")

# Load Data

In [2]:
dfRaw = pd.read_feather('../Data/FeatherData/dfAllDataLabeled.feather')

In [3]:
dfRaw = dfRaw.sort_values('UploadDate')

# Feature Engineering

In [4]:
dfRaw.shape

(1773, 21)

In [5]:
dfFeatures = pd.DataFrame(index=dfRaw.index)
dfFeatures['ViewCount'] = dfRaw['ViewCount']
dfFeatures['DaysSincePublication'] = dfRaw['DaysSincePublication']
dfFeatures['WatchList'] = dfRaw['WatchList']
dfFeatures['ViewsPerDay'] = dfFeatures['ViewCount'] / dfFeatures['DaysSincePublication']
dfFeatures = dfFeatures.drop('DaysSincePublication', axis=1)

# Split DataFrame into Training and Validation Dataset

In [6]:
dfRaw['UploadDate'].iloc[int(round(dfRaw.shape[0]/2,0))]

Timestamp('2019-11-27 00:00:00')

In [7]:
dateSplit = '2019-11-27'

maskTrain = (dfRaw['UploadDate'] < dateSplit)

maskVal = (dfRaw['UploadDate'] >= dateSplit)

Xtrain, Xval = dfFeatures[maskTrain].drop('WatchList', axis=1) , dfFeatures[maskVal].drop('WatchList', axis=1)
ytrain, yval = dfFeatures[maskTrain]['WatchList'], dfFeatures[maskVal]['WatchList']

In [8]:
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((885, 2), (888, 2), (885,), (888,))

# Text Features

In [9]:
titleTrain = dfRaw[maskTrain]['Title']
titleVal = dfRaw[maskVal]['Title']

titleVec = TfidfVectorizer(min_df=4, ngram_range=(1,2))
titleBowTrain = titleVec.fit_transform(titleTrain)
titleBowVal = titleVec.transform(titleVal)

In [10]:
titleBowTrain.shape

(885, 495)

In [11]:
titleBowTrain

<885x495 sparse matrix of type '<class 'numpy.float64'>'
	with 8627 stored elements in Compressed Sparse Row format>

In [12]:
XtrainWTitle = hstack([Xtrain, titleBowTrain])
XvalWTitle = hstack([Xval, titleBowVal])

In [13]:
XtrainWTitle.shape, XvalWTitle.shape

((885, 497), (888, 497))

# Model

## RandomForestClassifier

In [14]:
rf = RandomForestClassifier(n_estimators=1500, random_state=0, min_samples_leaf=2, class_weight='balanced', n_jobs=-1)
rf.fit(XtrainWTitle, ytrain)

RandomForestClassifier(class_weight='balanced', min_samples_leaf=2,
                       n_estimators=1500, n_jobs=-1, random_state=0)

In [16]:
pRf = rf.predict_proba(XvalWTitle)[:,1]

### Model Evaluate 

In [17]:
print(f'ap: {average_precision_score(yval, pRf)}, auc: {roc_auc_score(yval, pRf)}')

ap: 0.425697264360267, auc: 0.7845256438476778


In [19]:
# ap: 0.3521564476834772, auc: 0.7758089368258859 | min_samples_leaf=1
# ap: 0.3843011851670597, auc: 0.7815540391811577 | min_samples_leaf=2
# ap: 0.38551950447476385, auc: 0.7831168831168831 | n_estimators=1500
# ap: 0.40864867634952645, auc: 0.7861765353290777 | min_df=3
# ap: 0.4138684826920506, auc: 0.784668721109399 | ngram_range = (1,2)
# ap: 0.425697264360267, auc: 0.7845256438476778 | min_df=4 & ngram_range = (1,2)

## LGBMClassifier

In [20]:
modelLGBM = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=-1)
modelLGBM.fit(XtrainWTitle, ytrain)

LGBMClassifier(class_weight='balanced', random_state=0)

In [21]:
pLGBM = modelLGBM.predict_proba(XvalWTitle)[:,1]

### Model Evaluate 

In [22]:
print(f'ap: {average_precision_score(yval,pLGBM)}, auc: {roc_auc_score(yval, pLGBM)}')

ap: 0.3613860826479734, auc: 0.7497248514197665


### Bayesian Optimization

In [24]:
def tuneLGBM(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])
    
    titleVec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    titleBowTrain = titleVec.fit_transform(titleTrain)
    titleBowVal = titleVec.transform(titleVal)
    
    XtrainWTitle = hstack([Xtrain, titleBowTrain])
    XvalWTitle = hstack([Xval, titleBowVal])
    
    model = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                           min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree,
                           bagging_freq=1, n_estimators=n_estimators, random_state=0, class_weight='balanced', n_jobs=-1)
    
    model.fit(XtrainWTitle, ytrain)
    
    p = model.predict_proba(XvalWTitle)[:,1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval,p)



space = [(1e-3, 1e-1, 'log-uniform'), #lr
        (1, 10), #max_depth
        (1, 20), #min_child_samples
        (0.05, 1.0), #subsample
        (0.05, 1.0), #colsample_bytree
        (100, 1000), #n_estimetors
        (1, 5), #min_df
        (1, 5)] #ngram_range



result = forest_minimize(tuneLGBM, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.7713295179396874
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.7412
Function value obtained: -0.3746
Current minimum: -0.3746
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.7705040721989873
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.6712
Function value obtained: -0.3512
Current minimum: -0.3746
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsamp

In [25]:
result.x

[0.001477438108754858,
 3,
 10,
 0.16137221087324535,
 0.19593296243176872,
 966,
 3,
 5]

LGBMClassifier

[0.001477438108754858, 3, 10, 0.16137221087324535, 0.19593296243176872, 966, 3, 5]
ap: 0.4227, auc: 0.7806845696676206

## Model With Hyperparameter Tuning

In [68]:
params = [0.001477438108754858, #lr
 3, #max_depth
 10, #min_child_samples
 0.16137221087324535, #subsample
 0.19593296243176872, #colsample_bytree
 966, #n_estimetors
 2,  #min_df
 5] #ngram_range



lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]
min_df = params[6]
ngram_range = (1, params[7])

titleVec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
titleBowTrain = titleVec.fit_transform(titleTrain)
titleBowVal = titleVec.transform(titleVal)

XtrainWTitle = hstack([Xtrain, titleBowTrain])
XvalWTitle = hstack([Xval, titleBowVal])

model = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                       min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree,
                       bagging_freq=1, n_estimators=n_estimators, random_state=0, class_weight='balanced', n_jobs=-1)

model.fit(XtrainWTitle, ytrain)

pLgbm = model.predict_proba(XvalWTitle)[:,1]


print(f'ap: {average_precision_score(yval,pLgbm)}, auc: {roc_auc_score(yval, pLgbm)}')

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ap: 0.41114784456628284, auc: 0.7786154523442659


# Logistic Regression

### Data Preparation

In [69]:
XtrainWTitle2 = csr_matrix(XtrainWTitle.copy())
XvalWTitle2 = csr_matrix(XvalWTitle.copy())


scaler = MaxAbsScaler()

XtrainWTitle2 = scaler.fit_transform(XtrainWTitle2)
XvalWTitle2 = scaler.transform(XvalWTitle2)

In [70]:
XvalWTitle2.shape

(888, 2454)

### Model

In [71]:
model = LogisticRegression(C=0.01, n_jobs=-1, random_state=0)
model.fit(XtrainWTitle2, ytrain)

LogisticRegression(C=0.01, n_jobs=-1, random_state=0)

In [72]:
pLr = model.predict_proba(XvalWTitle2)[:,1]

In [73]:
print(f'ap: {average_precision_score(yval, pLr)}, auc: {roc_auc_score(yval, pLr)}')

ap: 0.3809915480394665, auc: 0.7685009905348889


ap: 0.3986324868099709, auc: 0.7804974686330618

# Ensamble

#### min_df > 3
- LGBM | ap: 0.42266615524449913, auc: 0.7806845696676206
- LR   | ap: 0.3986324868099709, auc: 0.7804974686330618

#### min_df > 2
- LGBM | ap: 0.41114784456628284, auc: 0.7786154523442659
- LR   | ap: 0.3809915480394665, auc: 0.7685009905348889

In [75]:
pd.DataFrame({'LR': pLr, 'LGBM': pLgbm}).corr()

,LR,LGBM
LR,1.000000,0.792928
LGBM,0.792928,1.000000


In [78]:
p = (0.30*pLgbm + 0.70*pLr)
print(f'ap: {average_precision_score(yval,p)}, auc: {roc_auc_score(yval, p)}')

ap: 0.4242863619606607, auc: 0.7871010345586616
